In [1]:
import importlib
import numpy as np
import pandas as pd
import utils

In [2]:
importlib.reload(utils)

<module 'utils' from '/home/fei/Documents/projects/lyrics/stacked_lstm_statefull/utils.py'>

In [3]:
from keras.models import Sequential, load_model
from keras.layers import Embedding, LSTM, Dense, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization, Bidirectional
from keras.preprocessing.sequence import pad_sequences
# from keras.optimizers import Adam

Using TensorFlow backend.


In [4]:
word2ind, ind2word = utils.load_index_word_map()

In [5]:
vocab_size = len(word2ind)

In [7]:
dataset = pd.read_csv('data/train_1003.csv')

In [8]:
dataset.head()

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,300
0,24,70,69,1,69,57,54,1,68,60,...,63,55,70,68,54,53,1,1,0,22
1,29,54,67,54,1,72,54,1,50,67,...,54,1,30,4,71,54,1,63,54,71
2,22,61,61,7,1,74,64,70,1,64,...,67,50,58,63,1,1,0,22,1,68
3,33,58,69,69,61,54,1,23,64,74,...,36,57,2,1,28,64,53,1,51,61
4,33,70,52,58,61,61,54,1,1,0,...,1,0,30,1,72,64,60,54,1,70


In [9]:
dataset.shape

(47101, 301)

In [13]:
batchs = pd.read_csv('data/batch_sizes.csv', index_col=0, header=None)

In [14]:
batchs.head()

,1
0,
848,79
734,78
739,75
850,74
860,74


In [15]:
emb_dim = 128
input_length = 300
batch_size = batchs.loc[1003, 1]

In [16]:
batch_size

67

In [17]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=input_length, batch_input_shape=(batch_size, input_length)))
model.add(Conv1D(filters=256, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, padding='same'))
model.add(Dropout(.2))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2, padding='same'))
model.add(Bidirectional(LSTM(100, return_sequences=True, dropout=.2, recurrent_dropout=.2, stateful=True)))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(100, dropout=.2, recurrent_dropout=.2, stateful=True)))
model.add(BatchNormalization())
model.add(Dropout(.2))
model.add(Dense(vocab_size, activation='softmax'))

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (67, 300, 128)            9728      
_________________________________________________________________
conv1d_1 (Conv1D)            (67, 300, 256)            131328    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (67, 150, 256)            0         
_________________________________________________________________
dropout_1 (Dropout)          (67, 150, 256)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (67, 150, 32)             32800     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (67, 75, 32)              0         
_________________________________________________________________
bidirectional_1 (Bidirection (67, 75, 200)             106400    
__________

In [15]:
model.set_weights(load_model('model_300.h5').get_weights())

In [19]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [20]:
for _ in range(10):
    print(_)
    model.reset_states()
    model.fit(dataset.iloc[:, :-1].values, dataset.iloc[:, [-1]].values, epochs=1, batch_size=batch_size)

Epoch 1/1
47101/47101 [==============================] - 122s - loss: 2.9340 - acc: 0.2802   
Epoch 1/1
47101/47101 [==============================] - 120s - loss: 2.1634 - acc: 0.3977   
Epoch 1/1
47101/47101 [==============================] - 120s - loss: 2.0367 - acc: 0.4266   
Epoch 1/1
47101/47101 [==============================] - 120s - loss: 1.9268 - acc: 0.4558   
Epoch 1/1
47101/47101 [==============================] - 120s - loss: 1.8749 - acc: 0.4681   
Epoch 1/1
47101/47101 [==============================] - 120s - loss: 1.8193 - acc: 0.4827   
Epoch 1/1
47101/47101 [==============================] - 119s - loss: 1.7676 - acc: 0.4945   
Epoch 1/1
47101/47101 [==============================] - 120s - loss: 1.7313 - acc: 0.5055   
Epoch 1/1
47101/47101 [==============================] - 119s - loss: 1.6903 - acc: 0.5160   
Epoch 1/1
47101/47101 [==============================] - 120s - loss: 1.6645 - acc: 0.5230   


In [21]:
# save model
model.save('model_300.h5')

In [22]:
pred_model = Sequential()
pred_model.add(Embedding(input_dim=vocab_size, output_dim=emb_dim, input_length=input_length, batch_input_shape=(1, input_length)))
pred_model.add(Conv1D(filters=256, kernel_size=4, padding='same', activation='relu'))
pred_model.add(MaxPooling1D(pool_size=2, padding='same'))
pred_model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
pred_model.add(MaxPooling1D(pool_size=2, padding='same'))
pred_model.add(Bidirectional(LSTM(100, return_sequences=True, dropout=.2, recurrent_dropout=.2, stateful=True)))
pred_model.add(BatchNormalization())
pred_model.add(Bidirectional(LSTM(100, dropout=.2, recurrent_dropout=.2, stateful=True)))
pred_model.add(BatchNormalization())
pred_model.add(Dense(vocab_size, activation='softmax'))

In [23]:
pred_model.set_weights(model.get_weights())

In [24]:
def implement(seed_text, maxlen=100, must_stop=2000):
    cleaned = list(seed_text)
    padded_input_tokens = utils.tokenise(word2ind, cleaned)
    res_tokens = [token for token in padded_input_tokens]
    while must_stop > 0:
        padded_input_tokens = pad_sequences([padded_input_tokens], maxlen=maxlen)
        probs = pred_model.predict(padded_input_tokens, batch_size=1)[0]
        first_n = np.argsort(probs)[-3:]
        probs = probs[first_n] / np.sum(probs[first_n])
        predicted = np.random.choice(first_n, p=probs)
        padded_input_tokens = padded_input_tokens[0].tolist()
        padded_input_tokens.append(predicted)
        res_tokens.append(predicted)
        must_stop -= 1
    detokenised = utils.detokenise(ind2word, res_tokens)
    return ''.join(detokenised)

In [26]:
res = implement('We are friends, good friends', 300, 200)

In [27]:
print(res)

We are friends, good friends  
This who the to me, wish a to me  
  
I know the and may  
That tal the and a to still to to  
  
The lead and me the for the  
I that to me  
That to too and a think  
That's a to me all a to to f
